In [1]:
import sys
sys.executable

'/home/konan94/miniconda3/envs/sptag/bin/python'

In [2]:
import SPTAG

import numpy as np
import pandas as pd
from PIL import Image
import csv
import os
from os import listdir
from os.path import isfile, join, splitext
import shutil
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import time

%matplotlib inline

In [3]:
n = 5
k = 5
r = 3

In [4]:
#arr = np.load('/home/konan94/project/faiss/pg_np_512d_vector_data.npy').astype('float32')   # must be float32 : https://github.com/microsoft/SPTAG/issues/11
#arr.shape

# generate test np
d = 2                           # dimension
nb = 100                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

arr = np.random.random((nb, d)).astype('float32')
#xb[:, 0] += np.arange(nb) / 1000.
#xq = np.random.random((nq, d)).astype('float32')
#xq[:, 0] += np.arange(nq) / 1000.



In [5]:
arr.shape

(100, 2)

In [7]:
x1 = arr[0:100, :]

In [8]:
q1 = arr[99:100, :]

In [10]:
def testBuild(algo, distmethod, x, out):
    i = SPTAG.AnnIndex(algo, 'Float', x.shape[1])
    i.SetBuildParam("NumberOfThreads", '4')
    i.SetBuildParam("DistCalcMethod", distmethod)
    ret = i.Build(x.tobytes(), x.shape[0])
    i.Save(out)
    
def testBuildWithMetaData(algo, distmethod, x, s, out):
    i = SPTAG.AnnIndex(algo, 'Float', x.shape[1])
    i.SetBuildParam("NumberOfThreads", '4')
    i.SetBuildParam("DistCalcMethod", distmethod)
    if i.BuildWithMetaData(x.tobytes(), s, x.shape[0]):
        i.Save(out)

def testSearch(index, q, k):
    j = SPTAG.AnnIndex.Load(index)
    for t in range(q.shape[0]):
        result = j.Search(q[t].tobytes(), k)
        print (result[0]) # ids
        print (result[1]) # distances

def testSearchWithMetaData(index, q, k):
    j = SPTAG.AnnIndex.Load(index)
    j.SetSearchParam("MaxCheck", '1024')
    for t in range(q.shape[0]):
        result = j.SearchWithMetaData(q[t].tobytes(), k)
        print (result[0]) # ids
        print (result[1]) # distances
        print (result[2]) # metadata

def testAdd(index, x, out, algo, distmethod):
    if index != None:
        i = SPTAG.AnnIndex.Load(index)
    else:
        i = SPTAG.AnnIndex(algo, 'Float', x.shape[1])
    i.SetBuildParam("NumberOfThreads", '4')
    i.SetBuildParam("DistCalcMethod", distmethod)
    if i.Add(x.tobytes(), x.shape[0]):
        i.Save(out)

def testAddWithMetaData(index, x, s, out, algo, distmethod):
    if index != None:
        i = SPTAG.AnnIndex.Load(index)
    else:
        i = SPTAG.AnnIndex(algo, 'Float', x.shape[1])
    i = SPTAG.AnnIndex(algo, 'Float', x.shape[1])
    i.SetBuildParam("NumberOfThreads", '4')
    i.SetBuildParam("DistCalcMethod", distmethod)
    if i.AddWithMetaData(x.tobytes(), s, x.shape[0]):
        i.Save(out)

def testDelete(index, x, out):
    i = SPTAG.AnnIndex.Load(index)
    ret = i.Delete(x.tobytes(), x.shape[0])
    print (ret)
    i.Save(out)

def Test(algo, distmethod):
    print ("dw test.............................")
    %time testBuild(algo, distmethod, x1, 'testindices')
    %time testSearch('testindices', q1, k)

In [11]:
# run test
if __name__ == '__main__':
    Test('BKT', 'L2')
    #Test('KDT', 'L2')

dw test.............................
CPU times: user 832 ms, sys: 10.3 ms, total: 842 ms
Wall time: 439 ms
[99, 28, 78, 9, 60]
[0.0, 0.0023052990436553955, 0.0026325455401092768, 0.004830984864383936, 0.007799973711371422]
CPU times: user 8.58 ms, sys: 3.86 ms, total: 12.4 ms
Wall time: 12.6 ms
